# AWS Sagemaker

In [2]:
import sagemaker
import sagemaker.amazon.common as smac
from sagemaker import get_execution_role
from sagemaker.predictor import json_deserializer

import boto3, csv, io, json
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix

In [3]:
!wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip

--2020-03-26 06:16:32--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  8.12MB/s    in 0.6s    

2020-03-26 06:16:33 (8.12 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]

Archive:  ml-100k.zip
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflat

In [4]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled
!head -10 ua.base.shuffled

/home/ec2-user/SageMaker/Engineering-Individual/ml-100k
187	196	4	879465507
211	520	4	879460096
622	1060	3	882671160
390	690	3	879693677
294	333	4	877818861
711	281	3	879995362
692	194	4	876953340
627	792	4	879530501
577	623	5	880475149
437	588	3	881002092


In [5]:
!head -10 ua.test

1	20	4	887431883
1	33	4	878542699
1	61	4	878542420
1	117	3	874965739
1	155	2	878542201
1	160	4	875072547
1	171	5	889751711
1	189	3	888732928
1	202	5	875072442
1	265	4	878542441


In [6]:
nbUsers=943
nbMovies=1682
nbFeatures=nbUsers+nbMovies+1000000

nbRatingsTrain=90570
nbRatingsTest=9430

In [7]:
moviesByUser = {}
for userId in range(nbUsers):
    moviesByUser[str(userId)]=[]
 
with open('ua.base.shuffled','r') as f:
    samples=csv.reader(f,delimiter='\t')
    for userId,movieId,rating,timestamp in samples:
        moviesByUser[str(int(userId)-1)].append(int(movieId)-1)

In [8]:
def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(nbUsers)+int(movieId)-1] = 1
            if int(rating) >= 4:
                Y.append(1)
            else:
                Y.append(0)
            line=line+1
            
    Y=np.array(Y).astype('float32')
    return X,Y

In [9]:
X_train, Y_train = loadDataset('ua.base.shuffled', nbRatingsTrain, nbFeatures)
X_test, Y_test = loadDataset('ua.test',nbRatingsTest,nbFeatures)

In [10]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (nbRatingsTrain, nbFeatures)
assert Y_train.shape == (nbRatingsTrain, )
zero_labels = np.count_nonzero(Y_train)
print("Training labels: %d zeros, %d ones" % (zero_labels, nbRatingsTrain-zero_labels))

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (nbRatingsTest, nbFeatures)
assert Y_test.shape  == (nbRatingsTest, )
zero_labels = np.count_nonzero(Y_test)
print("Test labels: %d zeros, %d ones" % (zero_labels, nbRatingsTest-zero_labels))

(90570, 1002625)
(90570,)
Training labels: 49906 zeros, 40664 ones
(9430, 1002625)
(9430,)
Test labels: 5469 zeros, 3961 ones


In [26]:
bucket = 'engineering0166'
prefix = 'sagemaker/fm-movielens'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train3')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test3')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [27]:
def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    buf.seek(0)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

s3://engineering0166/sagemaker/fm-movielens/train3/train.protobuf
s3://engineering0166/sagemaker/fm-movielens/test3/test.protobuf
Output: s3://engineering0166/sagemaker/fm-movielens/output


In [13]:
containers = {'us-west-2': '174872318107.dkr.ecr.us-west-2.amazonaws.com/factorization-machines:latest',
              'us-east-1': '382416733822.dkr.ecr.us-east-1.amazonaws.com/factorization-machines:latest',
              'us-east-2': '404615174143.dkr.ecr.us-east-2.amazonaws.com/factorization-machines:latest',
              'eu-west-1': '438346466558.dkr.ecr.eu-west-1.amazonaws.com/factorization-machines:latest'}

In [14]:
fm = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                   get_execution_role(), 
                                   train_instance_count=1, 
                                   train_instance_type='ml.c4.xlarge',
                                   output_path=output_prefix,
                                   sagemaker_session=sagemaker.Session())

fm.set_hyperparameters(feature_dim=nbFeatures,
                      predictor_type='binary_classifier',
                      mini_batch_size=1000,
                      num_factors=64,
                      epochs=100)

fm.fit({'train': train_data, 'test': test_data})

2020-03-26 06:17:13 Starting - Starting the training job...
2020-03-26 06:17:15 Starting - Launching requested ML instances.........
2020-03-26 06:18:49 Starting - Preparing the instances for training......
2020-03-26 06:20:10 Downloading - Downloading input data
2020-03-26 06:20:10 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[03/26/2020 06:20:26 INFO 139900517533504] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'b


2020-03-26 06:20:23 Training - Training image download completed. Training in progress.[2020-03-26 06:20:32.157] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 4, "duration": 1504, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:20:32 INFO 139900517533504] #quality_metric: host=algo-1, epoch=1, train binary_classification_accuracy <score>=0.561604395604
[03/26/2020 06:20:32 INFO 139900517533504] #quality_metric: host=algo-1, epoch=1, train binary_classification_cross_entropy <loss>=0.674028990358
[03/26/2020 06:20:32 INFO 139900517533504] #quality_metric: host=algo-1, epoch=1, train binary_f_1.000 <score>=0.713116640299
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1506.0889720916748, "sum": 1506.0889720916748, "min": 1506.0889720916748}}, "EndTime": 1585203632.158411, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203630.651921}

[03/26/2020 06:20:32 INFO 1399

[2020-03-26 06:20:42.627] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 18, "duration": 1439, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:20:42 INFO 139900517533504] #quality_metric: host=algo-1, epoch=8, train binary_classification_accuracy <score>=0.705714285714
[03/26/2020 06:20:42 INFO 139900517533504] #quality_metric: host=algo-1, epoch=8, train binary_classification_cross_entropy <loss>=0.620225349845
[03/26/2020 06:20:42 INFO 139900517533504] #quality_metric: host=algo-1, epoch=8, train binary_f_1.000 <score>=0.765379965306
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1441.8339729309082, "sum": 1441.8339729309082, "min": 1441.8339729309082}}, "EndTime": 1585203642.628269, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203641.185955}

[03/26/2020 06:20:42 INFO 139900517533504] #progress_metric: host=algo-1, completed 9 % of epochs
#metrics {"Metrics"

[2020-03-26 06:20:52.924] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 32, "duration": 1448, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:20:52 INFO 139900517533504] #quality_metric: host=algo-1, epoch=15, train binary_classification_accuracy <score>=0.72289010989
[03/26/2020 06:20:52 INFO 139900517533504] #quality_metric: host=algo-1, epoch=15, train binary_classification_cross_entropy <loss>=0.589738561609
[03/26/2020 06:20:52 INFO 139900517533504] #quality_metric: host=algo-1, epoch=15, train binary_f_1.000 <score>=0.766865437064
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1450.2949714660645, "sum": 1450.2949714660645, "min": 1450.2949714660645}}, "EndTime": 1585203652.924672, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203651.474001}

[03/26/2020 06:20:52 INFO 139900517533504] #progress_metric: host=algo-1, completed 16 % of epochs
#metrics {"Metri

[2020-03-26 06:20:58.737] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 40, "duration": 1439, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:20:58 INFO 139900517533504] #quality_metric: host=algo-1, epoch=19, train binary_classification_accuracy <score>=0.727472527473
[03/26/2020 06:20:58 INFO 139900517533504] #quality_metric: host=algo-1, epoch=19, train binary_classification_cross_entropy <loss>=0.578440970285
[03/26/2020 06:20:58 INFO 139900517533504] #quality_metric: host=algo-1, epoch=19, train binary_f_1.000 <score>=0.767476747675
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1441.417932510376, "sum": 1441.417932510376, "min": 1441.417932510376}}, "EndTime": 1585203658.737613, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203657.295798}

[03/26/2020 06:20:58 INFO 139900517533504] #progress_metric: host=algo-1, completed 20 % of epochs
#metrics {"Metrics

...

[2020-03-26 06:22:20.741] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 152, "duration": 1462, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:22:20 INFO 139900517533504] #quality_metric: host=algo-1, epoch=75, train binary_classification_accuracy <score>=0.747065934066
[03/26/2020 06:22:20 INFO 139900517533504] #quality_metric: host=algo-1, epoch=75, train binary_classification_cross_entropy <loss>=0.523199897179
[03/26/2020 06:22:20 INFO 139900517533504] #quality_metric: host=algo-1, epoch=75, train binary_f_1.000 <score>=0.776687914156
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1464.3688201904297, "sum": 1464.3688201904297, "min": 1464.3688201904297}}, "EndTime": 1585203740.741989, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203739.277201}

[03/26/2020 06:22:20 INFO 139900517533504] #progress_metric: host=algo-1, completed 76 % of epochs
#metrics {"Met

[2020-03-26 06:22:30.952] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 166, "duration": 1474, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:22:30 INFO 139900517533504] #quality_metric: host=algo-1, epoch=82, train binary_classification_accuracy <score>=0.748208791209
[03/26/2020 06:22:30 INFO 139900517533504] #quality_metric: host=algo-1, epoch=82, train binary_classification_cross_entropy <loss>=0.520085868081
[03/26/2020 06:22:30 INFO 139900517533504] #quality_metric: host=algo-1, epoch=82, train binary_f_1.000 <score>=0.777727118397
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1476.9670963287354, "sum": 1476.9670963287354, "min": 1476.9670963287354}}, "EndTime": 1585203750.952813, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203749.475411}

[03/26/2020 06:22:30 INFO 139900517533504] #progress_metric: host=algo-1, completed 83 % of epochs
#metrics {"Met

[2020-03-26 06:22:42.609] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 182, "duration": 1456, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:22:42 INFO 139900517533504] #quality_metric: host=algo-1, epoch=90, train binary_classification_accuracy <score>=0.749395604396
[03/26/2020 06:22:42 INFO 139900517533504] #quality_metric: host=algo-1, epoch=90, train binary_classification_cross_entropy <loss>=0.5168664752
[03/26/2020 06:22:42 INFO 139900517533504] #quality_metric: host=algo-1, epoch=90, train binary_f_1.000 <score>=0.77884773902
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1458.631992340088, "sum": 1458.631992340088, "min": 1458.631992340088}}, "EndTime": 1585203762.609881, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203761.150825}

[03/26/2020 06:22:42 INFO 139900517533504] #progress_metric: host=algo-1, completed 91 % of epochs
#metrics {"Metrics":

[2020-03-26 06:22:52.819] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 196, "duration": 1440, "num_examples": 91, "num_bytes": 5796480}
[03/26/2020 06:22:52 INFO 139900517533504] #quality_metric: host=algo-1, epoch=97, train binary_classification_accuracy <score>=0.750351648352
[03/26/2020 06:22:52 INFO 139900517533504] #quality_metric: host=algo-1, epoch=97, train binary_classification_cross_entropy <loss>=0.514268871936
[03/26/2020 06:22:52 INFO 139900517533504] #quality_metric: host=algo-1, epoch=97, train binary_f_1.000 <score>=0.779702106202
#metrics {"Metrics": {"update.time": {"count": 1, "max": 1442.3720836639404, "sum": 1442.3720836639404, "min": 1442.3720836639404}}, "EndTime": 1585203772.819568, "Dimensions": {"Host": "algo-1", "Operation": "training", "Algorithm": "factorization-machines"}, "StartTime": 1585203771.376816}

[03/26/2020 06:22:52 INFO 139900517533504] #progress_metric: host=algo-1, completed 98 % of epochs
#metrics {"Met


2020-03-26 06:23:01 Uploading - Uploading generated training model
2020-03-26 06:23:43 Completed - Training job completed
Training seconds: 232
Billable seconds: 232


In [15]:
fm_predictor = fm.deploy(instance_type='ml.t2.medium', initial_instance_count=1)

----------------!

In [33]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    #print (js)
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.serializer = fm_serializer
fm_predictor.deserializer = json_deserializer

In [34]:
def fm_serializer_sparse(data):
    js = {'instances': []}
    for row in data:
        shape = [row.shape[1]]
        keys = row.nonzero()[1].tolist()
        
        values_raw = row.data[0]
        values = [float(i) for i in values_raw] 
        #refer https://stackoverflow.com/questions/27050108/convert-numpy-type-to-python
        #the type in list value row is numpy,float32.
        
        
        data_row = {'data':{'features':{'keys':keys,
                                        'shape': shape,
                                        'values':values
                                       }
                           }
                   }
        #print(data_row)
        js['instances'].append(data_row)
        
    print (js)
    return json.dumps(js)

fm_predictor.content_type = 'application/json'
fm_predictor.deserializer = json_deserializer

In [35]:
import sys
sys.getsizeof(X_test[1000:1010].toarray())

40105112

In [36]:
fm_predictor.serializer = fm_serializer_sparse
result = fm_predictor.predict(X_test[1000:1010])
print(result)
print (Y_test[1000:1010])

{'instances': [{'data': {'features': {'keys': [100, 1164], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1194], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1223], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1224], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1246], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1311], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1347], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1413], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1538], 'shape': [1002625], 'values': [1.0, 1.0]}}}, {'data': {'features': {'keys': [100, 1771], 'shape': [1002625], 'values': [1.0, 1.0]}}}]}
{'predictions': [{'score': 0.6888535618782043, 'predicted_label': 1.0}, {'score': 0.1998045146

In [37]:
data = X_test[1000:1002].toarray()

In [38]:
def fm_serializer(data):
    js = {'instances': []}
    for row in data:
        js['instances'].append({'features': row.tolist()})
    # print js
    return json.dumps(js)


payload = fm_serializer(data)

runtime_client = boto3.client('sagemaker-runtime', region_name='us-east-2', 
                              aws_access_key_id='d7cd7e32-5412-432f-9128-ff98a5635496',
                              aws_secret_access_key='8PO/WH5ArS2w+mnUZ4OCLwEsSHiFiCTt0WS0kGl1'
                             )

In [39]:
# the endpoint traing on the 100k dataset and deployed to AWS
endpoint_name = 'factorization-machines-2020-03-26-06-17-12-845'
response = runtime_client.invoke_endpoint(EndpointName=endpoint_name,
                                          ContentType='application/json',
                                          Accept='application/json',
                                          Body=payload)

# the scoring returned for the datapoints for the user and movie
print(response)
print(response['Body'].read())

ConnectionClosedError: Connection was closed before we received a valid response from endpoint URL: "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/factorization-machines-2020-03-26-06-17-12-845/invocations".